In [347]:
import pandas as pd
%matplotlib inline
from sklearn import linear_model

In [348]:
# загрузка данных
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

In [349]:
# посмотрим на данные
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [350]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [351]:
df_train.shape

(891, 12)

In [352]:
# посомтрим на пропуски
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [353]:
# выбросим 2 записи, у которых нет Embarked
df_train = df_train[df_train['Embarked'].notnull()]

In [354]:
# объединим тестовую выборку и трейн, чтобы вместе предобработать
df_train['is_test'] = 0
df_test['is_test'] = 1

cnt = len(df_train)
df_y = df_train['Survived']
df_train.drop('Survived', axis=1, inplace=True)

df_train = pd.concat([df_train, df_test], axis=0)

In [355]:
# есть идея проверить и создать интересную фичу из имени и фамилии - там есть, как я понимаю, девечьи имена женщин, 
# поставим 1 если такое есть имя в скобочках и 0 если нет. 
df_train['Married'] = np.where((df_train['Name'].str.contains('\\(')) & (df_train['Name'].str.contains('\\)')), 1, 0)

In [356]:
# удалим из рассмотрения столбец с id пассажира, так как он не отражает связи с какими либо данными - просто порядковый номер
df_train.drop('PassengerId', axis=1, inplace = True)
df_train.drop('Ticket', axis=1, inplace = True)
df_train.drop('Cabin', axis=1, inplace = True)
df_train.drop('Name', axis=1, inplace = True)

In [287]:
# df_train = pd.get_dummies(df_train, columns=['Sex', 'Embarked'])

In [289]:
# from sklearn.preprocessing import Imputer
# fill_NaN = Imputer(missing_values=np.nan, strategy='median', axis=1)
# df_train = fill_NaN.fit_transform(df_train)

In [288]:
# посмотрим на примеры с пропуском Age
df_train

,Age,Fare,Parch,Pclass,SibSp,is_test,Married,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,3,1,0,0,0.0,1.0,0.0,0.0,1.0
1,38.0,71.2833,0,1,1,0,1,1.0,0.0,1.0,0.0,0.0
2,26.0,7.9250,0,3,0,0,0,1.0,0.0,0.0,0.0,1.0
3,35.0,53.1000,0,1,1,0,1,1.0,0.0,0.0,0.0,1.0
4,35.0,8.0500,0,3,0,0,0,0.0,1.0,0.0,0.0,1.0
5,NaN,8.4583,0,3,0,0,0,0.0,1.0,0.0,1.0,0.0
6,54.0,51.8625,0,1,0,0,0,0.0,1.0,0.0,0.0,1.0
7,2.0,21.0750,1,3,3,0,0,0.0,1.0,0.0,0.0,1.0
8,27.0,11.1333,2,3,0,0,1,1.0,0.0,0.0,0.0,1.0
9,14.0,30.0708,0,2,1,0,1,1.0,0.0,1.0,0.0,0.0


In [290]:
df_train['Age'].fillna((df_train['Age'].median()), inplace=True)
df_train['Fare'].fillna((df_train['Fare'].median()), inplace=True)

In [291]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_train)
df_train_scale = scaler.transform(df_train)

In [292]:
df_train_scale

array([[-0.58026066, -0.50183865, -0.44540733, ..., -0.51026115,
        -0.32231217,  0.65572717],
       [ 0.66229729,  0.7360229 , -0.44540733, ...,  1.95978079,
        -0.32231217, -1.52502451],
       [-0.26962117, -0.48878987, -0.44540733, ..., -0.51026115,
        -0.32231217,  0.65572717],
       ..., 
       [ 0.70112723, -0.50183865, -0.44540733, ..., -0.51026115,
        -0.32231217,  0.65572717],
       [-0.11430143, -0.48637343, -0.44540733, ..., -0.51026115,
        -0.32231217,  0.65572717],
       [-0.11430143, -0.20977212,  0.709647  , ...,  1.95978079,
        -0.32231217, -1.52502451]])

In [331]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

kfold = KFold(n_splits=5, shuffle=True, random_state=145)
from sklearn.ensemble import RandomForestClassifier
log = RandomForestClassifier(max_depth=6, n_estimators=12, max_features=5)

scores = cross_val_score(log,
                         df_train_scale[0:cnt], df_y, groups=None,
                         scoring = make_scorer(accuracy_score),
                         cv = kfold)

In [332]:

# LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, 
#                    fit_intercept=True, intercept_scaling=1, class_weight=None, 
#                    random_state=None, solver='liblinear', max_iter=100,
#                    multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)

In [333]:
np.average(scores)

0.81890433568209231

In [334]:
log.fit(df_train_scale[0:cnt], df_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features=5, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=12, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [335]:
predict = log.predict(df_train_scale[cnt:])

In [336]:
# grid = GridSearchCV(pipeline, param_grid, 
#                     scoring=make_scorer(r2_score),
#                     cv=kfold)

In [337]:
with open('submission0.txt', 'w') as out:
    out.write('PassengerId,Survived\n')
    for passenger, y in zip(df_test['PassengerId'], predict):
        out.write('%s,%s\n' % (passenger, y))